In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts
import custom_ts_length

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
input_num = 0 #this determines the data type via -> ['Wake', 'N1', 'N2','N3', 'REM'][input_num]
input_num = int(sys.argv[1]) #This will be input to the script

In [3]:
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/TS_Fresh_Raw_EEG/Data/'

In [ ]:
#generate all path names and class list(s) etc. 
channels = constants.channel_list
paths , class_list, sleep_night_list , sleep_type_list , participant_id_list = constants.generate_paths_and_info(blue_pebble = True)

In [11]:
folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
data_type = ['Wake', 'N1', 'N2','N3', 'REM'][input_num]
paths = joblib.load(folder + data_type + '_paths.pkl')

#Load each raw dataframe and convert into a row for the larger dataframe 
overall_dfs_rows = []

for path in paths['selected_paths']:
    data_epo = mne.read_epochs(path)
    df_full = data_epo.to_data_frame()

    #Convert the dataframe into a row
    new_row = {}
    for col in df_full.columns: #iterate through feature columns   
        series = df_full[col] #This gives you the series that you want to append to the new dataframe
        new_row[str(col)] = series
    
    overall_dfs_rows.append(new_row)

df_all_samples = pd.DataFrame(overall_dfs_rows)


Reading /user/work/ko20929/fif_files/Wakeful_data_2/HC_1101_Night 3_ec.fif ...
    Found the data of interest:
        t =       0.00 ...   29996.09 ms
        0 CTF compensation matrices available
Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_185691/3078738400.py:9: RuntimeWarning: This filename (/user/work/ko20929/fif_files/Wakeful_data_2/HC_1101_Night 3_ec.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data_epo = mne.read_epochs(path)


Reading /user/work/ko20929/fif_files/Wakeful_data_2/HC_1102_Night 2_ec.fif ...
    Found the data of interest:
        t =       0.00 ...   19996.09 ms
        0 CTF compensation matrices available
Not setting metadata
14 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_185691/3078738400.py:9: RuntimeWarning: This filename (/user/work/ko20929/fif_files/Wakeful_data_2/HC_1102_Night 2_ec.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data_epo = mne.read_epochs(path)


Reading /user/work/ko20929/fif_files/Wakeful_data_2/HC_1103_Night 2_ec.fif ...
    Found the data of interest:
        t =       0.00 ...   19996.09 ms
        0 CTF compensation matrices available
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated


...

/tmp/ipykernel_185691/3078738400.py:9: RuntimeWarning: This filename (/user/work/ko20929/fif_files/Wakeful_data_2/PD+RBD_3305_Night 2_ec.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data_epo = mne.read_epochs(path)


Reading /user/work/ko20929/fif_files/Wakeful_data_2/PD+RBD_3306_Night 1_ec.fif ...
    Found the data of interest:
        t =       0.00 ...   19996.09 ms
        0 CTF compensation matrices available
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_185691/3078738400.py:9: RuntimeWarning: This filename (/user/work/ko20929/fif_files/Wakeful_data_2/PD+RBD_3306_Night 1_ec.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data_epo = mne.read_epochs(path)


Reading /user/work/ko20929/fif_files/Wakeful_data_2/PD+RBD_3306_Night 2_ec.fif ...
    Found the data of interest:
        t =       0.00 ...   19996.09 ms
        0 CTF compensation matrices available
Not setting metadata
13 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_185691/3078738400.py:9: RuntimeWarning: This filename (/user/work/ko20929/fif_files/Wakeful_data_2/PD+RBD_3306_Night 2_ec.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data_epo = mne.read_epochs(path)


In [ ]:
#Choose how many minutes of the Raw EEG data you actually want to use
mins = 10
new_length = 256*60*mins #convert minutes to the number of data points
df_snipped = custom_ts_length.customise_df_ts_length(df_all_samples, new_length, impute = False)
df_snipped = df_snipped.drop(columns = ['time', 'condition', 'epoch'] )

In [ ]:
# convert into TS_Fresh format Dataframe so that TS Fresh statistics can be calculated
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(df_snipped, ts_cols = list(df_snipped.columns))

In [ ]:
setting = MinimalFCParameters()
setting = EfficientFCParameters()

In [ ]:
t1 = time.time()
extracted_ts_fresh_df = extract_features(subset_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=setting, n_jobs = 3)
t2 = time.time()

In [14]:
save_name = 'TS_Fresh_Stats_10_min_' + data_type + '.pkl'
save_name = 'Test_' + save_name
save_name = save_folder + save_name

In [15]:
joblib.dump(df_all_samples, save_name)

['/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/TS_Fresh_Raw_EEG/Data/Test_TS_Fresh_Stats_10_min_Wake.pkl']

In [9]:
time_taken = t2-t1
print(time_taken)

NameError: name 't2' is not defined